# Agenda

1. Pivot tables
2. Stack, unstack, melt
3. Joins
4. `filter` on columns
5. Cleaning data

In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame

In [2]:
df = DataFrame(np.random.randint(0, 1000, [4,5]),
              index=list('abcd'),
              columns=list('vwxyz'))
df

v    w    x    y    z
a  906  704  539  445  618
b  692  477  543  649  594
c  853  732  548  193  643
d  501  406  614  962  466

In [3]:
df.loc['e'] = [1,2,3,4,5]
df

v    w    x    y    z
a  906  704  539  445  618
b  692  477  543  649  594
c  853  732  548  193  643
d  501  406  614  962  466
e    1    2    3    4    5

In [4]:
new_row = {'w':20, 'x':30, 'z':50}
df.loc['f'] = new_row

In [5]:
df

v    w    x      y    z
a  906.0  704  539  445.0  618
b  692.0  477  543  649.0  594
c  853.0  732  548  193.0  643
d  501.0  406  614  962.0  466
e    1.0    2    3    4.0    5
f    NaN   20   30    NaN   50

In [6]:
df['w'] = [10, 20, 30, 40, 50, 60]
df

v   w    x      y    z
a  906.0  10  539  445.0  618
b  692.0  20  543  649.0  594
c  853.0  30  548  193.0  643
d  501.0  40  614  962.0  466
e    1.0  50    3    4.0    5
f    NaN  60   30    NaN   50

In [7]:
df['u'] = {'a':100, 'c':300, 'e':500}

In [8]:
df

v   w    x      y    z      u
a  906.0  10  539  445.0  618  100.0
b  692.0  20  543  649.0  594    NaN
c  853.0  30  548  193.0  643  300.0
d  501.0  40  614  962.0  466    NaN
e    1.0  50    3    4.0    5  500.0
f    NaN  60   30    NaN   50    NaN

# Grouping

If we have a data frame with:
- One categorical column
- One numeric column

We can use `groupby` to calculate an aggregation method once per category, for all numeric rows matching.

In [9]:
df = pd.read_csv('taxi.csv')
df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

In [10]:
# we can do a 2D grouping

df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

# Pivot table

A pivot table is a 2-dimensional grouping!  It's similar to our multi-index, but uses a table

- We need one categorical column -- this will be the index, for the rows
- A second categorical column - this will be for the columns
- A numeric column
- Aggregation method

In [11]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount')

passenger_count      0          1          2          3          4          5  \
VendorID                                                                        
1                25.57  16.941386  19.076807  19.002803  20.518657  20.466667   
2                  NaN  17.904989  17.855770  17.359076  17.927913  17.192379   

passenger_count          6  
VendorID                    
1                      NaN  
2                17.401355

In [12]:
df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

In [13]:
df.pivot(index='VendorID', columns='passenger_count', values='total_amount')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='mean')

passenger_count      0          1          2          3          4          5  \
VendorID                                                                        
1                25.57  16.941386  19.076807  19.002803  20.518657  20.466667   
2                  NaN  17.904989  17.855770  17.359076  17.927913  17.192379   

passenger_count          6  
VendorID                    
1                      NaN  
2                17.401355

In [16]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='max')

passenger_count      0       1       2      3      4       5      6
VendorID                                                           
1                36.39  252.35  102.35  73.55  73.84   27.30    NaN
2                  NaN  137.59  138.84  74.46  72.92  102.11  83.12

In [18]:
# what if we want both mean + standard deviation?
df.pivot_table(columns='VendorID', index='passenger_count', values='total_amount',
              aggfunc=['mean', 'std'])

mean                   std           
VendorID                 1          2          1          2
passenger_count                                            
0                25.570000        NaN  15.301791        NaN
1                16.941386  17.904989  15.369459  15.167490
2                19.076807  17.855770  16.049125  15.233530
3                19.002803  17.359076  15.389968  12.540837
4                20.518657  17.927913  15.845568  14.630713
5                20.466667  17.192379   6.751543  14.064202
6                      NaN  17.401355        NaN  13.363827

In [19]:
# what if we want to look at more than one value column?
df.pivot_table(columns='VendorID', index='passenger_count', values=['total_amount', 'trip_distance'],
              aggfunc='mean')

total_amount            trip_distance          
VendorID                   1          2             1         2
passenger_count                                                
0                  25.570000        NaN      4.600000       NaN
1                  16.941386  17.904989      2.956456  3.262967
2                  19.076807  17.855770      3.452027  3.328849
3                  19.002803  17.359076      3.588535  3.187189
4                  20.518657  17.927913      3.952239  3.440522
5                  20.466667  17.192379      4.933333  3.172553
6                        NaN  17.401355           NaN  3.170976

In [23]:
# what if we want to look at more than one category for columns?
df.pivot_table(columns=['VendorID', 'payment_type'], index='passenger_count', values='total_amount',
              aggfunc='mean')

VendorID                 1                                        2  \
payment_type             1          2          3       4          1   
passenger_count                                                       
0                25.570000        NaN        NaN     NaN        NaN   
1                19.673469  12.698253  13.215172  11.675  20.147390   
2                21.338436  16.006464  42.480000     NaN  20.623066   
3                19.225065  18.763418        NaN  20.800  18.548926   
4                24.138387  17.401667        NaN     NaN  21.539123   
5                      NaN  20.466667        NaN     NaN  19.634750   
6                      NaN        NaN        NaN     NaN  18.980288   

VendorID                              
payment_type             2    3    4  
passenger_count                       
0                      NaN  NaN  NaN  
1                14.510453 -5.3 -7.8  
2                13.905152  NaN  NaN  
3                15.586200  NaN  NaN  
4                14.378966  NaN  NaN  
5                13.225076  NaN  NaN  
6                15.361491  NaN  NaN

In [22]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [24]:
# everything can be a list!
df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

mean                                     \
                             total_amount            trip_distance             
VendorID                                1          2             1         2   
passenger_count payment_type                                                   
0               1               25.570000        NaN      4.600000       NaN   
1               1               19.673469  20.147390      3.346904  3.486413   
                2               12.698253  14.510453      2.358766  2.925338   
                3               13.215172  -5.300000      2.000000  0.465000   
                4               11.675000  -7.800000      1.925000  0.890000   
2               1               21.338436  20.623066      3.461350  3.751722   
                2               16.006464  13.905152      3.346388  2.725152   
                3               42.480000        NaN     11.700000       NaN   
3               1               19.225065  18.548926      3.110390  3.278926   
                2               18.763418  15.586200      4.054430  3.050500   
                4               20.800000        NaN      3.600000       NaN   
4               1               24.138387  21.539123      4.348387  4.059298   
                2               17.401667  14.378966      3.611111  2.832414   
5               1                     NaN  19.634750           NaN  3.518750   
                2               20.466667  13.225076      4.933333  2.610203   
6               1                     NaN  18.980288           NaN  3.178606   
                2                     NaN  15.361491           NaN  3.161118   

                                      std                                     
                             total_amount            trip_distance            
VendorID                                1          2             1         2  
passenger_count payment_type                                                  
0               1               15.301791        NaN      4.666905       NaN  
1               1               17.401710  16.330567      4.120243  4.238130  
                2               10.068471  12.397838      3.495789  3.973524  
                3               15.363039   2.828427      3.300000  0.657609  
                4                6.750000        NaN      1.960230       NaN  
2               1               17.341582  16.457943      4.038718  4.584112  
                2               13.427005  12.278545      4.502270  3.465173  
                3               27.061360        NaN      9.134002       NaN  
3               1               14.268286  13.073727      3.406595  3.529512  
                2               16.588856  11.539061      5.085722  3.363113  
                4                     NaN        NaN           NaN       NaN  
4               1               18.045435  17.046583      4.541429  4.831724  
                2               13.143542  10.806051      3.871048  4.002527  
5               1                     NaN  15.734323           NaN  4.261502  
                2                6.751543   9.604873      2.926317  3.393812  
6               1                     NaN  13.856405           NaN  3.499148  
                2                     NaN  12.447223           NaN  4.083165

In [25]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [29]:
# passenger_count 4
# payment_type 1

# function mean
# column total_amount
# vendor ID 2

pt.loc[
   (4, 1)   # row selector
,
   # column selector
   ('mean', 'total_amount', 2)
]

21.539122807017545

In [30]:
# I want payment type 1 for all passenger counts

pt.xs(1, level='payment_type')

mean                                             std  \
                total_amount            trip_distance           total_amount   
VendorID                   1          2             1         2            1   
passenger_count                                                                
0                  25.570000        NaN      4.600000       NaN    15.301791   
1                  19.673469  20.147390      3.346904  3.486413    17.401710   
2                  21.338436  20.623066      3.461350  3.751722    17.341582   
3                  19.225065  18.548926      3.110390  3.278926    14.268286   
4                  24.138387  21.539123      4.348387  4.059298    18.045435   
5                        NaN  19.634750           NaN  3.518750          NaN   
6                        NaN  18.980288           NaN  3.178606          NaN   

                                                    
                           trip_distance            
VendorID                 2             1         2  
passenger_count                                     
0                      NaN      4.666905       NaN  
1                16.330567      4.120243  4.238130  
2                16.457943      4.038718  4.584112  
3                13.073727      3.406595  3.529512  
4                17.046583      4.541429  4.831724  
5                15.734323           NaN  4.261502  
6                13.856405           NaN  3.499148

In [31]:
# we can use xs on columns, too!

pt.xs('trip_distance', level=1, axis='columns')

mean                 std          
VendorID                              1         2         1         2
passenger_count payment_type                                         
0               1              4.600000       NaN  4.666905       NaN
1               1              3.346904  3.486413  4.120243  4.238130
                2              2.358766  2.925338  3.495789  3.973524
                3              2.000000  0.465000  3.300000  0.657609
                4              1.925000  0.890000  1.960230       NaN
2               1              3.461350  3.751722  4.038718  4.584112
                2              3.346388  2.725152  4.502270  3.465173
                3             11.700000       NaN  9.134002       NaN
3               1              3.110390  3.278926  3.406595  3.529512
                2              4.054430  3.050500  5.085722  3.363113
                4              3.600000       NaN       NaN       NaN
4               1              4.348387  4.059298  4.541429  4.831724
                2              3.611111  2.832414  3.871048  4.002527
5               1                   NaN  3.518750       NaN  4.261502
                2              4.933333  2.610203  2.926317  3.393812
6               1                   NaN  3.178606       NaN  3.499148
                2                   NaN  3.161118       NaN  4.083165

# Exercise: Olympic pivot tables

1. Load the Olympic data (`olympic_athlete_events.csv`) into a data frame. Keep only the rows for the teams from : Israel, US, UK, France, India, China.
2. Create a pivot table of teams vs. sport, average height, where the year is equal to or after 1980, and the sport has `i` in its name.
3. Show a pivot table of teams vs. sport, min + max weight. 
4. Show a pivot table of teams vs. sport, min and max weight + height.
5. Show a pivot table of years vs. teams, number of competitors.  Retrieve the number for Israel in each year.

In [32]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [34]:
df = pd.read_csv('olympic_athlete_events.csv',
                usecols=['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'])
df.columns

Index(['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'], dtype='object')

In [39]:
# keep only rows from 1980-
df = df.loc[df['Year'] >= 1980]

In [43]:
# keep only from countries: Israel, US, UK, France, India, China

df = df.loc[df['Team'].isin(['United States', 'Israel', 'United Kingdom', 'France', 'India', 'China'])]
df.head(10)

Name   Age  Height  Weight           Team  Year  \
0         A Dijiang  24.0   180.0    80.0          China  1992   
1          A Lamusi  23.0   170.0    60.0          China  2012   
10  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
11  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
12  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
13  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
14  Per Knut Aaland  33.0   188.0    75.0  United States  1994   
15  Per Knut Aaland  33.0   188.0    75.0  United States  1994   
16  Per Knut Aaland  33.0   188.0    75.0  United States  1994   
17  Per Knut Aaland  33.0   188.0    75.0  United States  1994   

                   Sport  
0             Basketball  
1                   Judo  
10  Cross Country Skiing  
11  Cross Country Skiing  
12  Cross Country Skiing  
13  Cross Country Skiing  
14  Cross Country Skiing  
15  Cross Country Skiing  
16  Cross Country Skiing  
17  Cross Country Skiing

In [44]:
df['Team'].value_counts()

Team
United States    8858
France           5550
China            4874
India             759
Israel            474
Name: count, dtype: int64

In [49]:
# keep only sports with "i" in the name

df = df.loc[df['Sport'].str.contains('i')]
df.head()

Name   Age  Height  Weight           Team  Year  \
10  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
11  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
12  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
13  Per Knut Aaland  31.0   188.0    75.0  United States  1992   
14  Per Knut Aaland  33.0   188.0    75.0  United States  1994   

                   Sport  
10  Cross Country Skiing  
11  Cross Country Skiing  
12  Cross Country Skiing  
13  Cross Country Skiing  
14  Cross Country Skiing

In [50]:
# 2. Create a pivot table of teams vs. sport, average height, where the year is equal to or after 1980, and the sport has `i` in its name.

# index = Sport
# columns = Team
# values = Height
# aggregation method =  mean

df.pivot_table(index='Sport',
              columns='Team',
              values='Height',
              aggfunc='mean')




Team                            China      France       India      Israel  \
Sport                                                                       
Alpine Skiing              171.166667  173.445255  167.600000  181.000000   
Athletics                  174.985646  177.066901  171.186047  177.166667   
Badminton                  176.448980  173.650000  175.000000  171.000000   
Biathlon                   167.177215  173.540650         NaN         NaN   
Bobsleigh                         NaN  182.368421         NaN         NaN   
Boxing                     176.500000  175.033333  171.484848  171.000000   
Canoeing                   179.366972  179.234973         NaN  180.600000   
Cross Country Skiing       169.520408  172.902834  167.666667         NaN   
Curling                    172.411765  177.100000         NaN         NaN   
Cycling                    172.372340  176.200000         NaN  164.500000   
Diving                     162.678571  167.689655         NaN         NaN   
Equestrianism              176.181818  173.873563  171.400000         NaN   
Fencing                    180.378855  178.871429         NaN  167.285714   
Figure Skating             168.085714  166.644444         NaN  166.444444   
Freestyle Skiing           165.435897  172.031746         NaN         NaN   
Gymnastics                 156.984399  164.559157  151.000000  167.868852   
Nordic Combined                   NaN  177.369863         NaN         NaN   
Rhythmic Gymnastics        168.285714  164.750000         NaN  168.708333   
Rowing                     181.806061  185.420513  184.700000         NaN   
Sailing                    175.300000  178.947368  177.500000  175.510638   
Shooting                   171.385593  171.276471  172.756410  176.294118   
Short Track Speed Skating  170.074627  169.256757         NaN  186.000000   
Ski Jumping                175.750000  177.548387         NaN         NaN   
Snowboarding               167.692308  173.178082         NaN         NaN   
Speed Skating              171.711409  179.735294         NaN         NaN   
Swimming                   177.299401  182.399151  177.642857  178.593023   
Synchronized Swimming      168.944444  170.300000         NaN  162.250000   
Table Tennis               170.640625  172.906250  176.600000  167.000000   
Tennis                     172.942857  178.591549  178.588235  174.650000   
Trampolining               163.714286  171.333333         NaN         NaN   
Triathlon                  170.727273  174.038462         NaN  175.000000   
Weightlifting              163.253012  166.846154  155.722222  170.000000   
Wrestling                  171.740741  172.403509  168.955556  175.666667   

Team                       United States  
Sport                                     
Alpine Skiing                 174.090379  
Athletics                     178.065041  
Badminton                     170.818182  
Biathlon                      173.824645  
Bobsleigh                            NaN  
Boxing                        175.989583  
Canoeing                      178.350318  
Cross Country Skiing          173.645503  
Curling                       174.510638  
Cycling                       176.320346  
Diving                        168.582524  
Equestrianism                 173.544554  
Fencing                       178.852174  
Figure Skating                166.557377  
Freestyle Skiing              172.307692  
Gymnastics                    160.045902  
Nordic Combined               177.783133  
Rhythmic Gymnastics           165.142857  
Rowing                        185.046729  
Sailing                       178.260274  
Shooting                      173.233202  
Short Track Speed Skating     168.755556  
Ski Jumping                   179.214286  
Snowboarding                  172.470588  
Speed Skating                 173.213450  
Swimming                      182.943971  
Synchronized Swimming         167.354839  
Table Tennis                  171.298246  
Tennis        

In [52]:
# 3. Show a pivot table of teams vs. sport, min + max weight. 

# index = Sport
# columns = Team
# values = Weight
# aggfunc = ['min', 'max']

df.pivot_table(index='Sport',
              columns='Team',
              values='Weight',
              aggfunc=['min', 'max'])

min                                      max  \
Team                      China France India Israel United States  China   
Sport                                                                      
Alpine Skiing              60.0   47.0  54.0   74.0          53.0   75.0   
Athletics                  42.0   43.0  43.0   44.0          41.0  120.0   
Badminton                  52.0   55.0  59.0   61.0          52.0   81.0   
Biathlon                   52.0   52.0   NaN    NaN          48.0   71.0   
Bobsleigh                   NaN   70.0   NaN    NaN           NaN    NaN   
Boxing                     48.0   48.0  48.0   48.0          48.0  111.0   
Canoeing                   55.0   53.0   NaN   64.0          55.0   92.0   
Cross Country Skiing       49.0   50.0  62.0    NaN          48.0   83.0   
Curling                    60.0   62.0   NaN    NaN          52.0   80.0   
Cycling                    51.0   47.0   NaN   58.0          48.0   90.0   
Diving                     28.0   48.0   NaN    NaN          40.0   69.0   
Equestrianism              58.0   48.0  60.0    NaN          48.0   73.0   
Fencing                    55.0   50.0   NaN   49.0          48.0  108.0   
Figure Skating             41.0   35.0   NaN   45.0          36.0   76.0   
Freestyle Skiing           46.0   48.0   NaN    NaN          48.0   75.0   
Gymnastics                 30.0   35.0  47.0   45.0          34.0   72.0   
Nordic Combined             NaN   59.0   NaN    NaN          59.0    NaN   
Rhythmic Gymnastics        40.0   30.0   NaN   44.0          40.0   57.0   
Rowing                     42.0   45.0  67.0    NaN          42.0   98.0   
Sailing                    55.0   49.0  60.0   50.0          52.0  100.0   
Shooting                   40.0   42.0  46.0   60.0          51.0  110.0   
Short Track Speed Skating  53.0   46.0   NaN   84.0          50.0   78.0   
Ski Jumping                56.0   51.0   NaN    NaN          44.0   66.0   
Snowboarding               48.0   50.0   NaN    NaN          48.0   78.0   
Speed Skating              48.0   57.0   NaN    NaN          48.0   93.0   
Swimming                   50.0   52.0  58.0   52.0          48.0   98.0   
Synchronized Swimming      48.0   48.0   NaN   49.0          47.0  104.0   
Table Tennis               51.0   50.0  47.0   57.0          42.0   85.0   
Tennis                     55.0   54.0  57.0   52.0          55.0   85.0   
Trampolining               47.0   53.0   NaN    NaN          52.0   65.0   
Triathlon                  53.0   45.0   NaN   65.0          52.0   68.0   
Weightlifting              48.0   48.0  48.0   52.0          48.0  133.0   
Wrestling                  48.0   48.0  48.0   52.0          48.0  130.0   

                                                              
Team                      France  India Israel United States  
Sport                                                         
Alpine Skiing               98.0   71.0   74.0         102.0  
Athletics                  116.0  125.0   96.0         158.0  
Badminton                   80.0   85.0   61.0          91.0  
Biathlon                    80.0    NaN    NaN          84.0  
Bobsleigh                  123.0    NaN    NaN           NaN  
Boxing                     105.0   91.0   75.0         116.0  
Canoeing                    95.0    NaN   89.0          98.0  
Cross Country Skiing        81.0   74.0    NaN          86.0  
Curling                     80.0    NaN    NaN         102.0  
Cycling                    100.0    NaN   59.0         104.0  
Diving                      75.0    NaN    NaN          80.0  
Equestrianism              100.0   70.0    NaN          88.0  
Fencing                    100.0    NaN   80.0          98.0  
Figure Skating              82.0    NaN   84.0          86.0  
Freestyle Skiing            92.0    NaN    NaN          98.0  
Gymnastics                  74.0   47.0   77.0          75.0  
Nordic Combined             80.0    NaN    NaN          81.0  
Rhythmic Gymnastics         55.

In [53]:
# 4. Show a pivot table of teams vs. sport, min and max weight + height.

# index = 'Sport'
# columns = Team
# values = ['Weight', 'Height']
# aggfunc = ['min', 'max']

df.pivot_table(index='Sport',
              columns='Team',
              values=['Weight', 'Height'],
              aggfunc=['min', 'max'])

min                                            \
                          Height                                    Weight   
Team                       China France  India Israel United States  China   
Sport                                                                        
Alpine Skiing              160.0  154.0  162.0  181.0         160.0   60.0   
Athletics                  155.0  155.0  152.0  148.0         150.0   42.0   
Badminton                  161.0  164.0  160.0  171.0         157.0   52.0   
Biathlon                   160.0  160.0    NaN    NaN         152.0   52.0   
Bobsleigh                    NaN  167.0    NaN    NaN           NaN    NaN   
Boxing                     165.0  158.0  158.0  165.0         155.0   48.0   
Canoeing                   160.0  160.0    NaN  165.0         157.0   55.0   
Cross Country Skiing       159.0  155.0  155.0    NaN         157.0   49.0   
Curling                    162.0  174.0    NaN    NaN         155.0   60.0   
Cycling                    160.0  155.0    NaN  162.0         155.0   51.0   
Diving                     137.0  159.0    NaN    NaN         150.0   28.0   
Equestrianism              168.0  156.0  158.0    NaN         152.0   58.0   
Fencing                    164.0  162.0    NaN  161.0         157.0   55.0   
Figure Skating             153.0  148.0    NaN  152.0         147.0   41.0   
Freestyle Skiing           157.0  152.0    NaN    NaN         152.0   46.0   
Gymnastics                 136.0  140.0  151.0  155.0         139.0   30.0   
Nordic Combined              NaN  171.0    NaN    NaN         167.0    NaN   
Rhythmic Gymnastics        160.0  150.0    NaN  157.0         155.0   40.0   
Rowing                     150.0  150.0  180.0    NaN         142.0   42.0   
Sailing                    162.0  158.0  167.0  160.0         157.0   55.0   
Shooting                   152.0  146.0  157.0  167.0         155.0   40.0   
Short Track Speed Skating  160.0  156.0    NaN  186.0         152.0   53.0   
Ski Jumping                166.0  163.0    NaN    NaN         160.0   56.0   
Snowboarding               158.0  160.0    NaN    NaN         157.0   48.0   
Speed Skating              158.0  160.0    NaN    NaN         157.0   48.0   
Swimming                   162.0  163.0  163.0  159.0         160.0   50.0   
Synchronized Swimming      162.0  158.0    NaN  158.0         155.0   48.0   
Table Tennis               150.0  157.0  149.0  167.0         157.0   51.0   
Tennis                     164.0  165.0  165.0  159.0         160.0   55.0   
Trampolining               155.0  164.0    NaN    NaN         154.0   47.0   
Triathlon                  165.0  153.0    NaN  175.0         157.0   53.0   
Weightlifting              148.0  150.0  145.0  156.0         150.0   48.0   
Wrestling                  154.0  157.0  152.0  151.0         150.0   48.0   

                                                               max         \
                                                            Height          
Team                      France India Israel United States  China France   
Sport                                                                       
Alpine Skiing               47.0  54.0   74.0          53.0  180.0  191.0   
Athletics                   43.0  43.0   44.0          41.0  202.0  198.0   
Badminton                   55.0  59.0   61.0          52.0  190.0  186.0   
Biathlon                    52.0   NaN    NaN          48.0  175.0  188.0   
Bobsleigh                   70.0   NaN    NaN           NaN    NaN  198.0   
Boxing                      48.0  48.0   48.0          48.0  200.0  200.0   
Canoeing                    53.0   NaN   64.0          55.0  194.0  203.0   
Cross Country Skiing        50.0  62.0    NaN          48.0  188.0  186.0   
Curling                     62.0   NaN    NaN          52.0  185.0  183.0   
Cycling                     47.0   NaN   58.0          48.0  189.0  192.0   
Diving                      48.0   NaN    NaN          40.0  172

In [57]:
# 5. Show a pivot table of years vs. teams, number of competitors.
# Retrieve the number for Israel in each year.

# index = Year
# column = Team
# values = Name
# aggfunc = count

df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc='count').fillna(0).astype(int)

Team  China  France  India  Israel  United States
Year                                             
1980     38     230     36       0            106
1984    312     367     37      63            642
1988    348     452     36      29            712
1992    394     542     39      40            766
1994     43     131      0       1            187
1996    320     385     28      36            611
1998     68     123      0       3            212
2000    296     381     54      55            572
2002     83     138      0       3            231
2004    372     376     52      40            571
2006    140     154      4       2            242
2008    462     386     58      56            556
2010    102     190      2       4            249
2012    367     326     62      51            539
2014    105     188      2       6            274
2016    368     354     87      50            540

In [60]:
# 5. Show a pivot table of years vs. teams, number of competitors.
# Retrieve the number for Israel in each year.

# index = Year
# column = Team
# values = Name
# aggfunc = count

df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc=['min', 'max'])

min                              \
Team           China                      France   
Year                                               
1980    Bao Zhenghua                Alain Bondue   
1984    Bao Zhenghua  Agns Gosselin (-Tavernier)   
1988    Cai Jianming  Agns Gosselin (-Tavernier)   
1992  Bai Chongguang   Abdelghani "Ghani" Yalouz   
1994         Chen Lu                Alexis Blanc   
1996     Ba Yanchuan        Abdel Kader Chkhmani   
1998       An Yulong            Adrien Duvillard   
2000   Abudoureheman                Abdel Jebahi   
2002       An Yulong         Alexandre Rousselet   
2004       Ai Linuer        Abderrahim El Haouzy   
2006     An Weijiang         Alexandre Rousselet   
2008         Bai Xue                Adrien Hardy   
2010     Cai Xuetong                Adrien Thaux   
2012        Bai Anqi           Abdellatif Meftah   
2014        Ba Dexin     Adeline Baud (-Mugnier)   
2016       Ai Yanhan                 Adrien Bart   

                                                                     \
Team                               India                     Israel   
Year                                                                  
1980                  Adille Sumariwalla                        NaN   
1984                 Baljit Singh Kharab               Arie Gamliel   
1988  Alapurackal "Mercy" Kuttan-Mathews        Aharon Jacobashvili   
1992                        Abha Dhillan            Aleksey Bazarov   
1994                                 NaN           Michael Shmerkin   
1996                      Ambika Radhika             Alex Tripolski   
1998                                 NaN                Galit Chait   
2000                      Abhinav Bindra           Adi Maia Bichman   
2002                                 NaN               Olga Danilov   
2004                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2006                Bahadur Gurung Gupta             Mikail Renzhin   
2008                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2010                     Jamyang Namgial         Alexandra Zaretski   
2012                      Abhinav Bindra         Aleksandr Shatilov   
2014                     Himanshu Thakur  Alexei Yuryevich Bychenko   
2016                      Abhinav Bindra               Ageze Guadie   

                                                   max                        \
Team                      United States          China                France   
Year                                                                           
1980  Abigail E. "Abbi" Fisher (-Gould)  Zhao Weichang           Yvon Mougel   
1984                    Abdurrahim Kuzu   Zou Zhenxian         velyne Imbert   
1988  Abigail Knickerbocker "Abby" Peck    Zhuang Yong           velyne lien   
1992                   Aaron E. Pollock      Zou Sixin  ric Paul Alain Navet   
1994           Alva Ross "A J" Kitt, IV     Zhao Guona           tienne Gouy   
1996           Adam Christophe Saathoff      Zou Sixin           velyne lien   
1998                     Adam Hostetter    Zhao Hongbo        ric Le Chanony   
2000                Aaron Wells Peirsol         Zhu Yi          ric le Leuch   
2002                     Aelin Peterson  Zhang Xiaolei        Vincent Vittoz   
2004                Aaron Wells Peirsol    Zou Shiming  ric Paul Alain Navet   
2006         Abigail Jean "Abby" Larson  Zhang Zhongqi            milie Vina   
2008                       Aarik Wilson    Zou Shiming           lodie Gugan   
2010                     Alice McKennis      Zhou Yang            milie Vina   
2012        Abdihakim "Abdi" Abdirahman    Zou Shiming         tienne Daille   
2014                       Aaron Blunck      Zhou Yang        Xavier Bertoni   
2016                     Abbey Weitzeil       Zu Lijun         tienne Hubert   

                                                             \
Team                              India              Israel   
Year                           

In [63]:
# get data for Israel

pt = df.pivot_table(index='Year',
              columns='Team',
              values='Name',
              aggfunc=['min', 'max'])

pt.xs('Israel', level='Team', axis='columns')

min                 max
Year                                               
1980                        NaN                 NaN
1984               Arie Gamliel      Zehava Shmueli
1988        Aharon Jacobashvili           Yoel Sela
1992            Aleksey Bazarov           Yoel Sela
1994           Michael Shmerkin    Michael Shmerkin
1996             Alex Tripolski          Yoav Bruck
1998                Galit Chait   Sergey Sakhnovsky
2000           Adi Maia Bichman    Yuriy Yevseychyk
2002               Olga Danilov        Olga Danilov
2004  Aleksandr "Alex" Averbukh    Yuriy Yevseychyk
2006             Mikail Renzhin      Mikail Renzhin
2008  Aleksandr "Alex" Averbukh  Veronika Vitenberg
2010         Alexandra Zaretski      Roman Zaretski
2012         Aleksandr Shatilov        Zohar Zemiro
2014  Alexei Yuryevich Bychenko   Vladislav Bykanov
2016               Ageze Guadie   Zohar Hen Shikler

In [64]:
df = pd.read_csv('olympic_athlete_events.csv',
                usecols=['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'])
df.columns

Index(['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'], dtype='object')

In [65]:
# keep only rows from 1980-
df = df.loc[df['Year'] >= 1980]

In [69]:
# how do we get sports with just "i"

df.loc[df['Sport'].str.contains('i')]

Name   Age  Height  Weight         Team  Year  \
4           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
5           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
6           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
7           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
8           Christine Jacoba Aaftink  27.0   185.0    82.0  Netherlands  1994   
...                              ...   ...     ...     ...          ...   ...   
271109  Aleksandr Viktorovich Zyuzin  28.0   183.0    72.0       Russia  2004   
271112                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271113                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271114            Tomasz Ireneusz ya  30.0   185.0    96.0       Poland  1998   
271115            Tomasz Ireneusz ya  34.0   185.0    96.0       Poland  2002   

                Sport  
4       Speed Skating  
5       Speed Skating  
6       Speed Skating  
7       Speed Skating  
8       Speed Skating  
...               ...  
271109         Rowing  
271112    Ski Jumping  
271113    Ski Jumping  
271114      Bobsleigh  
271115      Bobsleigh  

[126438 rows x 7 columns]

In [70]:
# how do we get sports with "i" or "j"
# option 1: Use | (or)

df.loc[df['Sport'].str.contains('i')  | df['Sport'].str.contains('j')]

Name   Age  Height  Weight         Team  Year  \
4           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
5           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
6           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
7           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
8           Christine Jacoba Aaftink  27.0   185.0    82.0  Netherlands  1994   
...                              ...   ...     ...     ...          ...   ...   
271109  Aleksandr Viktorovich Zyuzin  28.0   183.0    72.0       Russia  2004   
271112                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271113                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271114            Tomasz Ireneusz ya  30.0   185.0    96.0       Poland  1998   
271115            Tomasz Ireneusz ya  34.0   185.0    96.0       Poland  2002   

                Sport  
4       Speed Skating  
5       Speed Skating  
6       Speed Skating  
7       Speed Skating  
8       Speed Skating  
...               ...  
271109         Rowing  
271112    Ski Jumping  
271113    Ski Jumping  
271114      Bobsleigh  
271115      Bobsleigh  

[126438 rows x 7 columns]

In [71]:
# option 2: Use a regular expression!

df.loc[df['Sport'].str.contains('[ij]')]

Name   Age  Height  Weight         Team  Year  \
4           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
5           Christine Jacoba Aaftink  21.0   185.0    82.0  Netherlands  1988   
6           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
7           Christine Jacoba Aaftink  25.0   185.0    82.0  Netherlands  1992   
8           Christine Jacoba Aaftink  27.0   185.0    82.0  Netherlands  1994   
...                              ...   ...     ...     ...          ...   ...   
271109  Aleksandr Viktorovich Zyuzin  28.0   183.0    72.0       Russia  2004   
271112                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271113                      Piotr ya  27.0   176.0    59.0       Poland  2014   
271114            Tomasz Ireneusz ya  30.0   185.0    96.0       Poland  1998   
271115            Tomasz Ireneusz ya  34.0   185.0    96.0       Poland  2002   

                Sport  
4       Speed Skating  
5       Speed Skating  
6       Speed Skating  
7       Speed Skating  
8       Speed Skating  
...               ...  
271109         Rowing  
271112    Ski Jumping  
271113    Ski Jumping  
271114      Bobsleigh  
271115      Bobsleigh  

[126438 rows x 7 columns]

In [75]:
# what if we want both i and j

df.loc[df['Sport'].str.contains('i.*s|s.*i')]

Name   Age  Height  Weight           Team  Year  \
10                Per Knut Aaland  31.0   188.0    75.0  United States  1992   
11                Per Knut Aaland  31.0   188.0    75.0  United States  1992   
12                Per Knut Aaland  31.0   188.0    75.0  United States  1992   
13                Per Knut Aaland  31.0   188.0    75.0  United States  1992   
14                Per Knut Aaland  33.0   188.0    75.0  United States  1994   
...                           ...   ...     ...     ...            ...   ...   
271099          Stavroula Zygouri  36.0   171.0    63.0         Greece  2004   
271102  Olesya Nikolayevna Zykina  19.0   171.0    64.0         Russia  2000   
271103  Olesya Nikolayevna Zykina  23.0   171.0    64.0         Russia  2004   
271114         Tomasz Ireneusz ya  30.0   185.0    96.0         Poland  1998   
271115         Tomasz Ireneusz ya  34.0   185.0    96.0         Poland  2002   

                       Sport  
10      Cross Country Skiing  
11      Cross Country Skiing  
12      Cross Country Skiing  
13      Cross Country Skiing  
14      Cross Country Skiing  
...                      ...  
271099             Wrestling  
271102             Athletics  
271103             Athletics  
271114             Bobsleigh  
271115             Bobsleigh  

[52415 rows x 7 columns]

In [76]:
pt

min                              \
Team           China                      France   
Year                                               
1980    Bao Zhenghua                Alain Bondue   
1984    Bao Zhenghua  Agns Gosselin (-Tavernier)   
1988    Cai Jianming  Agns Gosselin (-Tavernier)   
1992  Bai Chongguang   Abdelghani "Ghani" Yalouz   
1994         Chen Lu                Alexis Blanc   
1996     Ba Yanchuan        Abdel Kader Chkhmani   
1998       An Yulong            Adrien Duvillard   
2000   Abudoureheman                Abdel Jebahi   
2002       An Yulong         Alexandre Rousselet   
2004       Ai Linuer        Abderrahim El Haouzy   
2006     An Weijiang         Alexandre Rousselet   
2008         Bai Xue                Adrien Hardy   
2010     Cai Xuetong                Adrien Thaux   
2012        Bai Anqi           Abdellatif Meftah   
2014        Ba Dexin     Adeline Baud (-Mugnier)   
2016       Ai Yanhan                 Adrien Bart   

                                                                     \
Team                               India                     Israel   
Year                                                                  
1980                  Adille Sumariwalla                        NaN   
1984                 Baljit Singh Kharab               Arie Gamliel   
1988  Alapurackal "Mercy" Kuttan-Mathews        Aharon Jacobashvili   
1992                        Abha Dhillan            Aleksey Bazarov   
1994                                 NaN           Michael Shmerkin   
1996                      Ambika Radhika             Alex Tripolski   
1998                                 NaN                Galit Chait   
2000                      Abhinav Bindra           Adi Maia Bichman   
2002                                 NaN               Olga Danilov   
2004                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2006                Bahadur Gurung Gupta             Mikail Renzhin   
2008                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2010                     Jamyang Namgial         Alexandra Zaretski   
2012                      Abhinav Bindra         Aleksandr Shatilov   
2014                     Himanshu Thakur  Alexei Yuryevich Bychenko   
2016                      Abhinav Bindra               Ageze Guadie   

                                                   max                        \
Team                      United States          China                France   
Year                                                                           
1980  Abigail E. "Abbi" Fisher (-Gould)  Zhao Weichang           Yvon Mougel   
1984                    Abdurrahim Kuzu   Zou Zhenxian         velyne Imbert   
1988  Abigail Knickerbocker "Abby" Peck    Zhuang Yong           velyne lien   
1992                   Aaron E. Pollock      Zou Sixin  ric Paul Alain Navet   
1994           Alva Ross "A J" Kitt, IV     Zhao Guona           tienne Gouy   
1996           Adam Christophe Saathoff      Zou Sixin           velyne lien   
1998                     Adam Hostetter    Zhao Hongbo        ric Le Chanony   
2000                Aaron Wells Peirsol         Zhu Yi          ric le Leuch   
2002                     Aelin Peterson  Zhang Xiaolei        Vincent Vittoz   
2004                Aaron Wells Peirsol    Zou Shiming  ric Paul Alain Navet   
2006         Abigail Jean "Abby" Larson  Zhang Zhongqi            milie Vina   
2008                       Aarik Wilson    Zou Shiming           lodie Gugan   
2010                     Alice McKennis      Zhou Yang            milie Vina   
2012        Abdihakim "Abdi" Abdirahman    Zou Shiming         tienne Daille   
2014                       Aaron Blunck      Zhou Yang        Xavier Bertoni   
2016                     Abbey Weitzeil       Zu Lijun         tienne Hubert   

                                                             \
Team                              India              Israel   
Year                           

In [78]:
pt.T    # don't use () here!

Year                                            1980  \
    Team                                               
min China                               Bao Zhenghua   
    France                              Alain Bondue   
    India                         Adille Sumariwalla   
    Israel                                       NaN   
    United States  Abigail E. "Abbi" Fisher (-Gould)   
max China                              Zhao Weichang   
    France                               Yvon Mougel   
    India                     Tamil Selwan Muniswamy   
    Israel                                       NaN   
    United States      William Nourse "Billy" Taylor   

Year                                     1984  \
    Team                                        
min China                        Bao Zhenghua   
    France         Agns Gosselin (-Tavernier)   
    India                 Baljit Singh Kharab   
    Israel                       Arie Gamliel   
    United States             Abdurrahim Kuzu   
max China                        Zou Zhenxian   
    France                      velyne Imbert   
    India                         Vandana Rao   
    Israel                     Zehava Shmueli   
    United States     Willie James Smith, III   

Year                                             1988  \
    Team                                                
min China                                Cai Jianming   
    France                 Agns Gosselin (-Tavernier)   
    India          Alapurackal "Mercy" Kuttan-Mathews   
    Israel                        Aharon Jacobashvili   
    United States   Abigail Knickerbocker "Abby" Peck   
max China                                 Zhuang Yong   
    France                                velyne lien   
    India                                 Vinod Kumar   
    Israel                                  Yoel Sela   
    United States      Zina Lynna Garrison (-Jackson)   

Year                                         1992                      1994  \
    Team                                                                      
min China                          Bai Chongguang                   Chen Lu   
    France              Abdelghani "Ghani" Yalouz              Alexis Blanc   
    India                            Abha Dhillan                       NaN   
    Israel                        Aleksey Bazarov          Michael Shmerkin   
    United States                Aaron E. Pollock  Alva Ross "A J" Kitt, IV   
max China                               Zou Sixin                Zhao Guona   
    France                   ric Paul Alain Navet               tienne Gouy   
    India                             Vimal Kumar                       NaN   
    Israel                              Yoel Sela          Michael Shmerkin   
    United States  Zina Lynna Garrison (-Jackson)        Troy Robert Benson   

Year                                            1996                   1998  \
    Team                                                                      
min China                                Ba Yanchuan              An Yulong   
    France                      Abdel Kader Chkhmani       Adrien Duvillard   
    India                             Ambika Radhika                    NaN   
    Israel                            Alex Tripolski            Galit Chait   
    United States           Adam Christophe Saathoff         Adam Hostetter   
max China                                  Zou Sixin            Zhao Hongbo   
    France                               velyne lien         ric Le Chanony   
    India          Varalakshimi Pandimukkala Venkata                    NaN   
    Israel                                Yoav Bruck      Sergey Sakhnovsky   
    United States                    Zahir A. Raheem  William "Bill" Demong   

Year                                2000                            2002  \
    Team                                                                   
min

In [79]:
pt.transpose()    # same as pt.T, note that we need parentheses

Year                                            1980  \
    Team                                               
min China                               Bao Zhenghua   
    France                              Alain Bondue   
    India                         Adille Sumariwalla   
    Israel                                       NaN   
    United States  Abigail E. "Abbi" Fisher (-Gould)   
max China                              Zhao Weichang   
    France                               Yvon Mougel   
    India                     Tamil Selwan Muniswamy   
    Israel                                       NaN   
    United States      William Nourse "Billy" Taylor   

Year                                     1984  \
    Team                                        
min China                        Bao Zhenghua   
    France         Agns Gosselin (-Tavernier)   
    India                 Baljit Singh Kharab   
    Israel                       Arie Gamliel   
    United States             Abdurrahim Kuzu   
max China                        Zou Zhenxian   
    France                      velyne Imbert   
    India                         Vandana Rao   
    Israel                     Zehava Shmueli   
    United States     Willie James Smith, III   

Year                                             1988  \
    Team                                                
min China                                Cai Jianming   
    France                 Agns Gosselin (-Tavernier)   
    India          Alapurackal "Mercy" Kuttan-Mathews   
    Israel                        Aharon Jacobashvili   
    United States   Abigail Knickerbocker "Abby" Peck   
max China                                 Zhuang Yong   
    France                                velyne lien   
    India                                 Vinod Kumar   
    Israel                                  Yoel Sela   
    United States      Zina Lynna Garrison (-Jackson)   

Year                                         1992                      1994  \
    Team                                                                      
min China                          Bai Chongguang                   Chen Lu   
    France              Abdelghani "Ghani" Yalouz              Alexis Blanc   
    India                            Abha Dhillan                       NaN   
    Israel                        Aleksey Bazarov          Michael Shmerkin   
    United States                Aaron E. Pollock  Alva Ross "A J" Kitt, IV   
max China                               Zou Sixin                Zhao Guona   
    France                   ric Paul Alain Navet               tienne Gouy   
    India                             Vimal Kumar                       NaN   
    Israel                              Yoel Sela          Michael Shmerkin   
    United States  Zina Lynna Garrison (-Jackson)        Troy Robert Benson   

Year                                            1996                   1998  \
    Team                                                                      
min China                                Ba Yanchuan              An Yulong   
    France                      Abdel Kader Chkhmani       Adrien Duvillard   
    India                             Ambika Radhika                    NaN   
    Israel                            Alex Tripolski            Galit Chait   
    United States           Adam Christophe Saathoff         Adam Hostetter   
max China                                  Zou Sixin            Zhao Hongbo   
    France                               velyne lien         ric Le Chanony   
    India          Varalakshimi Pandimukkala Venkata                    NaN   
    Israel                                Yoav Bruck      Sergey Sakhnovsky   
    United States                    Zahir A. Raheem  William "Bill" Demong   

Year                                2000                            2002  \
    Team                                                                   
min

In [81]:
# stack -- take the 2nd layer of the column multi-index, and make it
# the 2nd layer of the row multi-index

pt.stack()

min  \
Year Team                                               
1980 China                               Bao Zhenghua   
     France                              Alain Bondue   
     India                         Adille Sumariwalla   
     United States  Abigail E. "Abbi" Fisher (-Gould)   
1984 China                               Bao Zhenghua   
...                                               ...   
2016 China                                  Ai Yanhan   
     France                               Adrien Bart   
     India                             Abhinav Bindra   
     Israel                              Ageze Guadie   
     United States                     Abbey Weitzeil   

                                              max  
Year Team                                          
1980 China                          Zhao Weichang  
     France                           Yvon Mougel  
     India                 Tamil Selwan Muniswamy  
     United States  William Nourse "Billy" Taylor  
1984 China                           Zou Zhenxian  
...                                           ...  
2016 China                               Zu Lijun  
     France                         tienne Hubert  
     India                         Yogeshwar Dutt  
     Israel                     Zohar Hen Shikler  
     United States          Zachery "Zach" Ziemek  

[76 rows x 2 columns]

In [82]:
pt.stack().unstack()

min                              \
Team           China                      France   
Year                                               
1980    Bao Zhenghua                Alain Bondue   
1984    Bao Zhenghua  Agns Gosselin (-Tavernier)   
1988    Cai Jianming  Agns Gosselin (-Tavernier)   
1992  Bai Chongguang   Abdelghani "Ghani" Yalouz   
1994         Chen Lu                Alexis Blanc   
1996     Ba Yanchuan        Abdel Kader Chkhmani   
1998       An Yulong            Adrien Duvillard   
2000   Abudoureheman                Abdel Jebahi   
2002       An Yulong         Alexandre Rousselet   
2004       Ai Linuer        Abderrahim El Haouzy   
2006     An Weijiang         Alexandre Rousselet   
2008         Bai Xue                Adrien Hardy   
2010     Cai Xuetong                Adrien Thaux   
2012        Bai Anqi           Abdellatif Meftah   
2014        Ba Dexin     Adeline Baud (-Mugnier)   
2016       Ai Yanhan                 Adrien Bart   

                                                                     \
Team                               India                     Israel   
Year                                                                  
1980                  Adille Sumariwalla                        NaN   
1984                 Baljit Singh Kharab               Arie Gamliel   
1988  Alapurackal "Mercy" Kuttan-Mathews        Aharon Jacobashvili   
1992                        Abha Dhillan            Aleksey Bazarov   
1994                                 NaN           Michael Shmerkin   
1996                      Ambika Radhika             Alex Tripolski   
1998                                 NaN                Galit Chait   
2000                      Abhinav Bindra           Adi Maia Bichman   
2002                                 NaN               Olga Danilov   
2004                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2006                Bahadur Gurung Gupta             Mikail Renzhin   
2008                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2010                     Jamyang Namgial         Alexandra Zaretski   
2012                      Abhinav Bindra         Aleksandr Shatilov   
2014                     Himanshu Thakur  Alexei Yuryevich Bychenko   
2016                      Abhinav Bindra               Ageze Guadie   

                                                   max                        \
Team                      United States          China                France   
Year                                                                           
1980  Abigail E. "Abbi" Fisher (-Gould)  Zhao Weichang           Yvon Mougel   
1984                    Abdurrahim Kuzu   Zou Zhenxian         velyne Imbert   
1988  Abigail Knickerbocker "Abby" Peck    Zhuang Yong           velyne lien   
1992                   Aaron E. Pollock      Zou Sixin  ric Paul Alain Navet   
1994           Alva Ross "A J" Kitt, IV     Zhao Guona           tienne Gouy   
1996           Adam Christophe Saathoff      Zou Sixin           velyne lien   
1998                     Adam Hostetter    Zhao Hongbo        ric Le Chanony   
2000                Aaron Wells Peirsol         Zhu Yi          ric le Leuch   
2002                     Aelin Peterson  Zhang Xiaolei        Vincent Vittoz   
2004                Aaron Wells Peirsol    Zou Shiming  ric Paul Alain Navet   
2006         Abigail Jean "Abby" Larson  Zhang Zhongqi            milie Vina   
2008                       Aarik Wilson    Zou Shiming           lodie Gugan   
2010                     Alice McKennis      Zhou Yang            milie Vina   
2012        Abdihakim "Abdi" Abdirahman    Zou Shiming         tienne Daille   
2014                       Aaron Blunck      Zhou Yang        Xavier Bertoni   
2016                     Abbey Weitzeil       Zu Lijun         tienne Hubert   

                                                             \
Team                              India              Israel   
Year                           

In [84]:
pt.stack()

min  \
Year Team                                               
1980 China                               Bao Zhenghua   
     France                              Alain Bondue   
     India                         Adille Sumariwalla   
     United States  Abigail E. "Abbi" Fisher (-Gould)   
1984 China                               Bao Zhenghua   
...                                               ...   
2016 China                                  Ai Yanhan   
     France                               Adrien Bart   
     India                             Abhinav Bindra   
     Israel                              Ageze Guadie   
     United States                     Abbey Weitzeil   

                                              max  
Year Team                                          
1980 China                          Zhao Weichang  
     France                           Yvon Mougel  
     India                 Tamil Selwan Muniswamy  
     United States  William Nourse "Billy" Taylor  
1984 China                           Zou Zhenxian  
...                                           ...  
2016 China                               Zu Lijun  
     France                         tienne Hubert  
     India                         Yogeshwar Dutt  
     Israel                     Zohar Hen Shikler  
     United States          Zachery "Zach" Ziemek  

[76 rows x 2 columns]

In [85]:
pt.stack(level=0)

Team               China                      France  \
Year                                                   
1980 max   Zhao Weichang                 Yvon Mougel   
     min    Bao Zhenghua                Alain Bondue   
1984 max    Zou Zhenxian               velyne Imbert   
     min    Bao Zhenghua  Agns Gosselin (-Tavernier)   
1988 max     Zhuang Yong                 velyne lien   
     min    Cai Jianming  Agns Gosselin (-Tavernier)   
1992 max       Zou Sixin        ric Paul Alain Navet   
     min  Bai Chongguang   Abdelghani "Ghani" Yalouz   
1994 max      Zhao Guona                 tienne Gouy   
     min         Chen Lu                Alexis Blanc   
1996 max       Zou Sixin                 velyne lien   
     min     Ba Yanchuan        Abdel Kader Chkhmani   
1998 max     Zhao Hongbo              ric Le Chanony   
     min       An Yulong            Adrien Duvillard   
2000 max          Zhu Yi                ric le Leuch   
     min   Abudoureheman                Abdel Jebahi   
2002 max   Zhang Xiaolei              Vincent Vittoz   
     min       An Yulong         Alexandre Rousselet   
2004 max     Zou Shiming        ric Paul Alain Navet   
     min       Ai Linuer        Abderrahim El Haouzy   
2006 max   Zhang Zhongqi                  milie Vina   
     min     An Weijiang         Alexandre Rousselet   
2008 max     Zou Shiming                 lodie Gugan   
     min         Bai Xue                Adrien Hardy   
2010 max       Zhou Yang                  milie Vina   
     min     Cai Xuetong                Adrien Thaux   
2012 max     Zou Shiming               tienne Daille   
     min        Bai Anqi           Abdellatif Meftah   
2014 max       Zhou Yang              Xavier Bertoni   
     min        Ba Dexin     Adeline Baud (-Mugnier)   
2016 max        Zu Lijun               tienne Hubert   
     min       Ai Yanhan                 Adrien Bart   

Team                                   India                     Israel  \
Year                                                                      
1980 max              Tamil Selwan Muniswamy                        NaN   
     min                  Adille Sumariwalla                        NaN   
1984 max                         Vandana Rao             Zehava Shmueli   
     min                 Baljit Singh Kharab               Arie Gamliel   
1988 max                         Vinod Kumar                  Yoel Sela   
     min  Alapurackal "Mercy" Kuttan-Mathews        Aharon Jacobashvili   
1992 max                         Vimal Kumar                  Yoel Sela   
     min                        Abha Dhillan            Aleksey Bazarov   
1994 max                                 NaN           Michael Shmerkin   
     min                                 NaN           Michael Shmerkin   
1996 max   Varalakshimi Pandimukkala Venkata                 Yoav Bruck   
     min                      Ambika Radhika             Alex Tripolski   
1998 max                                 NaN          Sergey Sakhnovsky   
     min                                 NaN                Galit Chait   
2000 max                     Vinita Tripathi           Yuriy Yevseychyk   
     min                      Abhinav Bindra           Adi Maia Bichman   
2002 max                                 NaN               Olga Danilov   
     min                                 NaN               Olga Danilov   
2004 max                      Yogeshwar Dutt           Yuriy Yevseychyk   
     min                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2006 max                          Neha Ahuja             Mikail Renzhin   
     min                Bahadur Gurung Gupta             Mikail Renzhin   
2008 max                      Yogeshwar Dutt         Veronika Vitenberg   
     min                      Abhinav Bindra  Aleksandr "Alex" Averbukh   
2010 max                        Tashi Lundup             Roman Zaretski   
     min                     Jamyang Namgial         Alexandra Zaretsk

# Exercise: Pivot tables + stacking + unstacking with taxis

1. Read `taxi.csv` into a data frame.
2. Create a pivot table with the rows being the passenger_count, the columns will be VendorID, the values will be the trip_distance and total_amount, and we want to calculate min, max, and mean.
3. Move the min/max/mean to be on the rows, rather than the columns.
4. Move the trip_distance and total_amount to be on the rows, rather than the columns.
5. After you've done 4, move the passenger_count to be on the columns.

In [86]:
df = pd.read_csv('taxi.csv')
df.head()

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2015-06-02 11:19:29   2015-06-02 11:47:52                1   
1         2  2015-06-02 11:19:30   2015-06-02 11:27:56                1   
2         2  2015-06-02 11:19:31   2015-06-02 11:30:30                1   
3         2  2015-06-02 11:19:31   2015-06-02 11:39:02                1   
4         1  2015-06-02 11:19:32   2015-06-02 11:32:49                1   

   trip_distance  pickup_longitude  pickup_latitude  RateCodeID  \
0           1.63        -73.954430        40.764141           1   
1           0.46        -73.971443        40.758942           1   
2           0.87        -73.978111        40.738434           1   
3           2.13        -73.945892        40.773529           1   
4           1.40        -73.979088        40.776772           1   

  store_and_fwd_flag  dropoff_longitude  dropoff_latitude  payment_type  \
0                  N         -73.974754         40.754093             2   
1                  N         -73.978539         40.761909             1   
2                  N         -73.990273         40.745438             1   
3                  N         -73.971527         40.760330             1   
4                  N         -73.982162         40.758999             2   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0         17.0    0.0      0.5        0.00           0.0   
1          6.5    0.0      0.5        1.00           0.0   
2          8.0    0.0      0.5        2.20           0.0   
3         13.5    0.0      0.5        2.86           0.0   
4          9.5    0.0      0.5        0.00           0.0   

   improvement_surcharge  total_amount  
0                    0.3         17.80  
1                    0.3          8.30  
2                    0.3         11.00  
3                    0.3         17.16  
4                    0.3         10.30

In [87]:
# 2. Create a pivot table with the rows being the passenger_count, 
# the columns will be VendorID, the values will be the 
# trip_distance and total_amount, and we want to calculate min, max, and mean.

# index = passenger_count
# columns = VendorID
# values = ['trip_distance', 'total_amount']
# aggfunc = ['min', 'max', 'mean']

pt = df.pivot_table(index='passenger_count',
                   columns='VendorID',
                   values=['trip_distance', 'total_amount'],
                   aggfunc=['min', 'max', 'mean'])
pt

min                                  max          \
                total_amount      trip_distance      total_amount           
VendorID                   1    2             1    2            1       2   
passenger_count                                                             
0                      14.75  NaN           1.3  NaN        36.39     NaN   
1                       0.30 -7.8           0.0  0.0       252.35  137.59   
2                       0.30  2.3           0.0  0.0       102.35  138.84   
3                       4.80  4.8           0.0  0.0        73.55   74.46   
4                       5.80  3.3           0.0  0.0        73.84   72.92   
5                      13.80  3.3           3.0  0.0        27.30  102.11   
6                        NaN  3.3           NaN  0.0          NaN   83.12   

                                             mean                           \
                trip_distance        total_amount            trip_distance   
VendorID                    1      2            1          2             1   
passenger_count                                                              
0                         7.9    NaN    25.570000        NaN      4.600000   
1                        64.6  35.51    16.941386  17.904989      2.956456   
2                        26.0  29.78    19.076807  17.855770      3.452027   
3                        18.0  21.80    19.002803  17.359076      3.588535   
4                        19.3  22.67    20.518657  17.927913      3.952239   
5                         8.3  24.97    20.466667  17.192379      4.933333   
6                         NaN  23.81          NaN  17.401355           NaN   

                           
                           
VendorID                2  
passenger_count            
0                     NaN  
1                3.262967  
2                3.328849  
3                3.187189  
4                3.440522  
5                3.172553  
6                3.170976

In [88]:
# 3. Move the min/max/mean to be on the rows, rather than the columns.

pt.stack(level=0)

total_amount             trip_distance           
VendorID                        1           2             1          2
passenger_count                                                       
0               max     36.390000         NaN      7.900000        NaN
                mean    25.570000         NaN      4.600000        NaN
                min     14.750000         NaN      1.300000        NaN
1               max    252.350000  137.590000     64.600000  35.510000
                mean    16.941386   17.904989      2.956456   3.262967
                min      0.300000   -7.800000      0.000000   0.000000
2               max    102.350000  138.840000     26.000000  29.780000
                mean    19.076807   17.855770      3.452027   3.328849
                min      0.300000    2.300000      0.000000   0.000000
3               max     73.550000   74.460000     18.000000  21.800000
                mean    19.002803   17.359076      3.588535   3.187189
                min      4.800000    4.800000      0.000000   0.000000
4               max     73.840000   72.920000     19.300000  22.670000
                mean    20.518657   17.927913      3.952239   3.440522
                min      5.800000    3.300000      0.000000   0.000000
5               max     27.300000  102.110000      8.300000  24.970000
                mean    20.466667   17.192379      4.933333   3.172553
                min     13.800000    3.300000      3.000000   0.000000
6               max           NaN   83.120000           NaN  23.810000
                mean          NaN   17.401355           NaN   3.170976
                min           NaN    3.300000           NaN   0.000000

In [91]:
# 4. Move the trip_distance and total_amount to be on the rows, rather than the columns.

pt = pt.stack(level=1)
pt

max               mean               min  \
VendorID                            1       2          1          2      1   
passenger_count                                                              
0               total_amount    36.39     NaN  25.570000        NaN  14.75   
                trip_distance    7.90     NaN   4.600000        NaN   1.30   
1               total_amount   252.35  137.59  16.941386  17.904989   0.30   
                trip_distance   64.60   35.51   2.956456   3.262967   0.00   
2               total_amount   102.35  138.84  19.076807  17.855770   0.30   
                trip_distance   26.00   29.78   3.452027   3.328849   0.00   
3               total_amount    73.55   74.46  19.002803  17.359076   4.80   
                trip_distance   18.00   21.80   3.588535   3.187189   0.00   
4               total_amount    73.84   72.92  20.518657  17.927913   5.80   
                trip_distance   19.30   22.67   3.952239   3.440522   0.00   
5               total_amount    27.30  102.11  20.466667  17.192379  13.80   
                trip_distance    8.30   24.97   4.933333   3.172553   3.00   
6               total_amount      NaN   83.12        NaN  17.401355    NaN   
                trip_distance     NaN   23.81        NaN   3.170976    NaN   

                                    
VendorID                         2  
passenger_count                     
0               total_amount   NaN  
                trip_distance  NaN  
1               total_amount  -7.8  
                trip_distance  0.0  
2               total_amount   2.3  
                trip_distance  0.0  
3               total_amount   4.8  
                trip_distance  0.0  
4               total_amount   3.3  
                trip_distance  0.0  
5               total_amount   3.3  
                trip_distance  0.0  
6               total_amount   3.3  
                trip_distance  0.0

In [92]:
# 5. After you've done 4, move the passenger_count to be on the columns.

pt.unstack(level=0)

max                                                      \
VendorID             1                                           2           
passenger_count      0       1       2      3      4     5   6   0       1   
total_amount     36.39  252.35  102.35  73.55  73.84  27.3 NaN NaN  137.59   
trip_distance     7.90   64.60   26.00  18.00  19.30   8.3 NaN NaN   35.51   

                         ...  min                                              
VendorID                 ...    1             2                                
passenger_count       2  ...    4     5   6   0    1    2    3    4    5    6  
total_amount     138.84  ...  5.8  13.8 NaN NaN -7.8  2.3  4.8  3.3  3.3  3.3  
trip_distance     29.78  ...  0.0   3.0 NaN NaN  0.0  0.0  0.0  0.0  0.0  0.0  

[2 rows x 42 columns]

In [93]:
print(pt.unstack(level=0))

                   max                                                      \
VendorID             1                                           2           
passenger_count      0       1       2      3      4     5   6   0       1   
total_amount     36.39  252.35  102.35  73.55  73.84  27.3 NaN NaN  137.59   
trip_distance     7.90   64.60   26.00  18.00  19.30   8.3 NaN NaN   35.51   

                         ...  min                                              
VendorID                 ...    1             2                                
passenger_count       2  ...    4     5   6   0    1    2    3    4    5    6  
total_amount     138.84  ...  5.8  13.8 NaN NaN -7.8  2.3  4.8  3.3  3.3  3.3  
trip_distance     29.78  ...  0.0   3.0 NaN NaN  0.0  0.0  0.0  0.0  0.0  0.0  

[2 rows x 42 columns]


In [96]:
with open('/tmp/mydata.html', 'w') as f:
    f.write(pt.unstack(level=0).to_html())

In [97]:
help(df.to_html)

Help on method to_html in module pandas.core.frame:

to_html(buf: 'FilePath | WriteBuffer[str] | None' = None, columns: 'Sequence[Level] | None' = None, col_space: 'ColspaceArgType | None' = None, header: 'bool | Sequence[str]' = True, index: 'bool' = True, na_rep: 'str' = 'NaN', formatters: 'FormattersType | None' = None, float_format: 'FloatFormatType | None' = None, sparsify: 'bool | None' = None, index_names: 'bool' = True, justify: 'str | None' = None, max_rows: 'int | None' = None, max_cols: 'int | None' = None, show_dimensions: 'bool | str' = False, decimal: 'str' = '.', bold_rows: 'bool' = True, classes: 'str | list | tuple | None' = None, escape: 'bool' = True, notebook: 'bool' = False, border: 'int | bool | None' = None, table_id: 'str | None' = None, render_links: 'bool' = False, encoding: 'str | None' = None) -> 'str | None' method of pandas.core.frame.DataFrame instance
    Render a DataFrame as an HTML table.
    
    Parameters
    ----------
    buf : str, Path or Strin

In [104]:
df.pivot_table(index='passenger_count',
                   columns='VendorID',
                   values='total_amount', aggfunc='min', margins=True)

VendorID             1    2    All
passenger_count                   
0                14.75  NaN  14.75
1                 0.30 -7.8  -7.80
2                 0.30  2.3   0.30
3                 4.80  4.8   4.80
4                 5.80  3.3   3.30
5                13.80  3.3   3.30
6                  NaN  3.3   3.30
All               0.30 -7.8  -7.80

In [105]:
df.pivot_table(index='passenger_count',
                   columns='VendorID',
                   values='total_amount', aggfunc=['min', 'max'], margins=True)

min                 max                
VendorID             1    2    All       1       2     All
passenger_count                                           
0                14.75  NaN  14.75   36.39     NaN   36.39
1                 0.30 -7.8  -7.80  252.35  137.59  252.35
2                 0.30  2.3   0.30  102.35  138.84  138.84
3                 4.80  4.8   4.80   73.55   74.46   74.46
4                 5.80  3.3   3.30   73.84   72.92   73.84
5                13.80  3.3   3.30   27.30  102.11  102.11
6                  NaN  3.3   3.30     NaN   83.12   83.12
All               0.30 -7.8  -7.80  252.35  138.84  252.35

In [106]:
products_df = DataFrame([{'product_id':23, 'name':'computer',
                          'wholesale_price': 500,
                          'retail_price':1000,
                          'department':'electronics'},
                        {'product_id':96, 'name':'Python Workout',
                         'wholesale_price': 35,
                         'retail_price':75, 'department':'books'},
                        {'product_id':97, 'name':'Pandas Workout',
                         'wholesale_price': 35,
                         'retail_price':75, 'department':'books'},
                        {'product_id':15, 'name':'banana',
                         'wholesale_price': 0.5,
                         'retail_price':1, 'department':'food'},
                        {'product_id':87, 'name':'sandwich',
                        'wholesale_price': 3,
                         'retail_price':5, 'department': 'food'},
                        ])


In [107]:
sales_df = DataFrame([{'product_id': 23, 'date':'2021-August-10',
                       'quantity':1},
                     {'product_id': 96, 'date':'2021-August-10',
                       'quantity':5},
                     {'product_id': 15, 'date':'2021-August-10',
                       'quantity':3},
                     {'product_id': 87, 'date':'2021-August-10',
                       'quantity':2},
                     {'product_id': 15, 'date':'2021-August-11',
                       'quantity':1},
                     {'product_id': 96, 'date':'2021-August-11',
                       'quantity':1},
                     {'product_id': 23, 'date':'2021-August-11',
                       'quantity':2},
                     {'product_id': 87, 'date':'2021-August-12',
                       'quantity':2},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':6},
                     {'product_id': 97, 'date':'2021-August-12',
                       'quantity':1},
                     {'product_id': 87, 'date':'2021-August-13',
                       'quantity':2},
                     {'product_id': 23, 'date':'2021-August-13',
                       'quantity':1},
                     {'product_id': 15, 'date':'2021-August-14',
                       'quantity':2}
                     ])

In [108]:
products_df

product_id            name  wholesale_price  retail_price   department
0          23        computer            500.0          1000  electronics
1          96  Python Workout             35.0            75        books
2          97  Pandas Workout             35.0            75        books
3          15          banana              0.5             1         food
4          87        sandwich              3.0             5         food

In [109]:
sales_df

product_id            date  quantity
0           23  2021-August-10         1
1           96  2021-August-10         5
2           15  2021-August-10         3
3           87  2021-August-10         2
4           15  2021-August-11         1
5           96  2021-August-11         1
6           23  2021-August-11         2
7           87  2021-August-12         2
8           97  2021-August-12         6
9           97  2021-August-12         1
10          87  2021-August-13         2
11          23  2021-August-13         1
12          15  2021-August-14         2

In [116]:
# how much money did we make from sales?

sales_df.set_index('product_id').join(products_df.set_index('product_id')).groupby('department')['retail_price'].sum()

department
books           300
electronics    3000
food             18
Name: retail_price, dtype: int64

In [117]:
!ls *oecd*

oecd_locations.csv  oecd_tourism.csv


In [118]:
!cat oecd_locations.csv

﻿AUS,Australia
AUT,Austria
BEL,Belgium
CAN,Canada
DNK,Denmark
FIN,Finland
FRA,France
DEU,Germany
HUN,Hungary
ITA,Italy
JPN,Japan
KOR,Korea
GBR,United Kingdom
USA,United States
BRA,Brazil
ISR,Israel


In [121]:
!head -20 oecd_tourism.csv

﻿"LOCATION","INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2008",31159.8,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2009",29980.7,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2010",35165.5,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2011",38710.1,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2012",38003.7,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2013",36965,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2014",38047.9,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2015",36226,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2016",39082.3,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2017",43959.1,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2018",47259.8,
"AUS","TOUR_REC_EXP","INT_REC","USD","A","2019",47930.9,
"AUS","TOUR_REC_EXP","INT-EXP","USD","A","2008",27620,
"AUS","TOUR_REC_EXP","INT-EXP","USD","A","2009",25629.6,
"AUS","TOUR_REC_EXP","INT-EXP","USD","A","2010",31916.5,
"AUS","TOUR_REC_EXP","INT-EXP","USD","A","2011",39381.5,
"AUS","TOU

# Exercise: OECD tourism

1. Read the OECD data into two different data frames.
2. What five countries received/earned the most, on average per year, from tourism?

In [ ]:
sales_df.set_index('product_id').join(
    bproducts_df.set_index('product_id')).groupby('department')['retail_price'].sum()

In [122]:
!ls *oecd*

oecd_locations.csv  oecd_tourism.csv


In [131]:
tourism_df = pd.read_csv('oecd_tourism.csv', index_col='LOCATION')
tourism_df.head()

INDICATOR  SUBJECT MEASURE FREQUENCY  TIME    Value  Flag Codes
LOCATION                                                                    
AUS       TOUR_REC_EXP  INT_REC     USD         A  2008  31159.8         NaN
AUS       TOUR_REC_EXP  INT_REC     USD         A  2009  29980.7         NaN
AUS       TOUR_REC_EXP  INT_REC     USD         A  2010  35165.5         NaN
AUS       TOUR_REC_EXP  INT_REC     USD         A  2011  38710.1         NaN
AUS       TOUR_REC_EXP  INT_REC     USD         A  2012  38003.7         NaN

In [134]:
locations_df = pd.read_csv('oecd_locations.csv', header=None,
                          names=['LOCATION', 'name'],
                          index_col='LOCATION')
locations_df.head()

name
LOCATION           
AUS       Australia
AUT         Austria
BEL         Belgium
CAN          Canada
DNK         Denmark

In [138]:
df = tourism_df.join(locations_df)
df.head(15)

INDICATOR  SUBJECT MEASURE FREQUENCY  TIME    Value  Flag Codes  \
LOCATION                                                                       
AUS       TOUR_REC_EXP  INT_REC     USD         A  2008  31159.8         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2009  29980.7         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2010  35165.5         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2011  38710.1         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2012  38003.7         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2013  36965.0         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2014  38047.9         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2015  36226.0         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2016  39082.3         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2017  43959.1         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2018  47259.8         NaN   
AUS       TOUR_REC_EXP  INT_REC     USD         A  2019  47930.9         NaN   
AUS       TOUR_REC_EXP  INT-EXP     USD         A  2008  27620.0         NaN   
AUS       TOUR_REC_EXP  INT-EXP     USD         A  2009  25629.6         NaN   
AUS       TOUR_REC_EXP  INT-EXP     USD         A  2010  31916.5         NaN   

               name  
LOCATION             
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia  
AUS       Australia

In [143]:
# 2. What five countries received/earned the most, on average per year, from tourism?

df.loc[
     df['SUBJECT'] == 'INT_REC',   # row selector
   ['name', 'Value']
].groupby('name')['Value'].mean().sort_values(ascending=False).head(5)

name
United States     201613.500000
France             65063.335727
Germany            53408.570636
United Kingdom     51752.090909
Italy              44930.211545
Name: Value, dtype: float64

# Filtering by rows + columns

In [144]:
df = pd.read_csv('taxi.csv')
df.head()

VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2015-06-02 11:19:29   2015-06-02 11:47:52                1   
1         2  2015-06-02 11:19:30   2015-06-02 11:27:56                1   
2         2  2015-06-02 11:19:31   2015-06-02 11:30:30                1   
3         2  2015-06-02 11:19:31   2015-06-02 11:39:02                1   
4         1  2015-06-02 11:19:32   2015-06-02 11:32:49                1   

   trip_distance  pickup_longitude  pickup_latitude  RateCodeID  \
0           1.63        -73.954430        40.764141           1   
1           0.46        -73.971443        40.758942           1   
2           0.87        -73.978111        40.738434           1   
3           2.13        -73.945892        40.773529           1   
4           1.40        -73.979088        40.776772           1   

  store_and_fwd_flag  dropoff_longitude  dropoff_latitude  payment_type  \
0                  N         -73.974754         40.754093             2   
1                  N         -73.978539         40.761909             1   
2                  N         -73.990273         40.745438             1   
3                  N         -73.971527         40.760330             1   
4                  N         -73.982162         40.758999             2   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0         17.0    0.0      0.5        0.00           0.0   
1          6.5    0.0      0.5        1.00           0.0   
2          8.0    0.0      0.5        2.20           0.0   
3         13.5    0.0      0.5        2.86           0.0   
4          9.5    0.0      0.5        0.00           0.0   

   improvement_surcharge  total_amount  
0                    0.3         17.80  
1                    0.3          8.30  
2                    0.3         11.00  
3                    0.3         17.16  
4                    0.3         10.30

In [146]:
# one column -- easy!
df['passenger_count']

0       1
1       1
2       1
3       1
4       1
       ..
9994    1
9995    1
9996    1
9997    6
9998    1
Name: passenger_count, Length: 9999, dtype: int64

In [147]:
# two columns
df[['passenger_count', 'trip_distance']]

passenger_count  trip_distance
0                   1           1.63
1                   1           0.46
2                   1           0.87
3                   1           2.13
4                   1           1.40
...               ...            ...
9994                1           2.70
9995                1           4.50
9996                1           5.59
9997                6           1.54
9998                1           5.80

[9999 rows x 2 columns]

In [150]:
# we want all columns with "pickup" in the column name

# list comprehension
df[[one_column
 for one_column in df.columns
 if 'pickup' in one_column]]

tpep_pickup_datetime  pickup_longitude  pickup_latitude
0     2015-06-02 11:19:29        -73.954430        40.764141
1     2015-06-02 11:19:30        -73.971443        40.758942
2     2015-06-02 11:19:31        -73.978111        40.738434
3     2015-06-02 11:19:31        -73.945892        40.773529
4     2015-06-02 11:19:32        -73.979088        40.776772
...                   ...               ...              ...
9994  2015-06-01 00:12:59        -73.947792        40.814972
9995  2015-06-01 00:12:59        -74.004066        40.747818
9996  2015-06-01 00:13:00        -73.994377        40.766102
9997  2015-06-01 00:13:02        -73.978302        40.748531
9998  2015-06-01 00:13:04        -73.983215        40.726414

[9999 rows x 3 columns]

In [152]:
# df.filter -- give it the name we want to find!

df.filter(like='pickup')

tpep_pickup_datetime  pickup_longitude  pickup_latitude
0     2015-06-02 11:19:29        -73.954430        40.764141
1     2015-06-02 11:19:30        -73.971443        40.758942
2     2015-06-02 11:19:31        -73.978111        40.738434
3     2015-06-02 11:19:31        -73.945892        40.773529
4     2015-06-02 11:19:32        -73.979088        40.776772
...                   ...               ...              ...
9994  2015-06-01 00:12:59        -73.947792        40.814972
9995  2015-06-01 00:12:59        -74.004066        40.747818
9996  2015-06-01 00:13:00        -73.994377        40.766102
9997  2015-06-01 00:13:02        -73.978302        40.748531
9998  2015-06-01 00:13:04        -73.983215        40.726414

[9999 rows x 3 columns]

In [154]:
#nwhat if  I want pickup + dropoff

df.filter(regex='pickup|dropoff')

tpep_pickup_datetime tpep_dropoff_datetime  pickup_longitude  \
0     2015-06-02 11:19:29   2015-06-02 11:47:52        -73.954430   
1     2015-06-02 11:19:30   2015-06-02 11:27:56        -73.971443   
2     2015-06-02 11:19:31   2015-06-02 11:30:30        -73.978111   
3     2015-06-02 11:19:31   2015-06-02 11:39:02        -73.945892   
4     2015-06-02 11:19:32   2015-06-02 11:32:49        -73.979088   
...                   ...                   ...               ...   
9994  2015-06-01 00:12:59   2015-06-01 00:24:18        -73.947792   
9995  2015-06-01 00:12:59   2015-06-01 00:28:16        -74.004066   
9996  2015-06-01 00:13:00   2015-06-01 00:37:25        -73.994377   
9997  2015-06-01 00:13:02   2015-06-01 00:19:10        -73.978302   
9998  2015-06-01 00:13:04   2015-06-01 00:36:33        -73.983215   

      pickup_latitude  dropoff_longitude  dropoff_latitude  
0           40.764141         -73.974754         40.754093  
1           40.758942         -73.978539         40.761909  
2           40.738434         -73.990273         40.745438  
3           40.773529         -73.971527         40.760330  
4           40.776772         -73.982162         40.758999  
...               ...                ...               ...  
9994        40.814972         -73.973358         40.783638  
9995        40.747818         -73.953758         40.779285  
9996        40.766102         -73.903206         40.750546  
9997        40.748531         -73.989166         40.762852  
9998        40.726414         -73.924133         40.701645  

[9999 rows x 6 columns]

In [155]:
# what about the rows

df = df.set_index('tpep_pickup_datetime')
df.head()

VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2015-06-02 11:19:29          2   2015-06-02 11:47:52                1   
2015-06-02 11:19:30          2   2015-06-02 11:27:56                1   
2015-06-02 11:19:31          2   2015-06-02 11:30:30                1   
2015-06-02 11:19:31          2   2015-06-02 11:39:02                1   
2015-06-02 11:19:32          1   2015-06-02 11:32:49                1   

                      trip_distance  pickup_longitude  pickup_latitude  \
tpep_pickup_datetime                                                     
2015-06-02 11:19:29            1.63        -73.954430        40.764141   
2015-06-02 11:19:30            0.46        -73.971443        40.758942   
2015-06-02 11:19:31            0.87        -73.978111        40.738434   
2015-06-02 11:19:31            2.13        -73.945892        40.773529   
2015-06-02 11:19:32            1.40        -73.979088        40.776772   

                      RateCodeID store_and_fwd_flag  dropoff_longitude  \
tpep_pickup_datetime                                                     
2015-06-02 11:19:29            1                  N         -73.974754   
2015-06-02 11:19:30            1                  N         -73.978539   
2015-06-02 11:19:31            1                  N         -73.990273   
2015-06-02 11:19:31            1                  N         -73.971527   
2015-06-02 11:19:32            1                  N         -73.982162   

                      dropoff_latitude  payment_type  fare_amount  extra  \
tpep_pickup_datetime                                                       
2015-06-02 11:19:29          40.754093             2         17.0    0.0   
2015-06-02 11:19:30          40.761909             1          6.5    0.0   
2015-06-02 11:19:31          40.745438             1          8.0    0.0   
2015-06-02 11:19:31          40.760330             1         13.5    0.0   
2015-06-02 11:19:32          40.758999             2          9.5    0.0   

                      mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                      
2015-06-02 11:19:29       0.5        0.00           0.0   
2015-06-02 11:19:30       0.5        1.00           0.0   
2015-06-02 11:19:31       0.5        2.20           0.0   
2015-06-02 11:19:31       0.5        2.86           0.0   
2015-06-02 11:19:32       0.5        0.00           0.0   

                      improvement_surcharge  total_amount  
tpep_pickup_datetime                                       
2015-06-02 11:19:29                     0.3         17.80  
2015-06-02 11:19:30                     0.3          8.30  
2015-06-02 11:19:31                     0.3         11.00  
2015-06-02 11:19:31                     0.3         17.16  
2015-06-02 11:19:32                     0.3         10.30

In [156]:
df.filter(like='11:19', axis='rows')

VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2015-06-02 11:19:29          2   2015-06-02 11:47:52                1   
2015-06-02 11:19:30          2   2015-06-02 11:27:56                1   
2015-06-02 11:19:31          2   2015-06-02 11:30:30                1   
2015-06-02 11:19:31          2   2015-06-02 11:39:02                1   
2015-06-02 11:19:32          1   2015-06-02 11:32:49                1   
...                        ...                   ...              ...   
2015-06-01 00:11:19          2   2015-06-01 00:16:22                1   
2015-06-01 00:11:19          1   2015-06-01 00:15:12                1   
2015-06-01 00:11:19          2   2015-06-01 00:58:23                1   
2015-06-01 00:11:19          1   2015-06-01 00:14:31                1   
2015-06-01 00:11:19          1   2015-06-01 00:29:53                2   

                      trip_distance  pickup_longitude  pickup_latitude  \
tpep_pickup_datetime                                                     
2015-06-02 11:19:29            1.63        -73.954430        40.764141   
2015-06-02 11:19:30            0.46        -73.971443        40.758942   
2015-06-02 11:19:31            0.87        -73.978111        40.738434   
2015-06-02 11:19:31            2.13        -73.945892        40.773529   
2015-06-02 11:19:32            1.40        -73.979088        40.776772   
...                             ...               ...              ...   
2015-06-01 00:11:19            1.79        -73.979408        40.761662   
2015-06-01 00:11:19            0.20        -74.004921        40.737457   
2015-06-01 00:11:19           20.61        -73.781776        40.644661   
2015-06-01 00:11:19            0.90        -73.956131        40.767998   
2015-06-01 00:11:19            8.30        -73.993980        40.750996   

                      RateCodeID store_and_fwd_flag  dropoff_longitude  \
tpep_pickup_datetime                                                     
2015-06-02 11:19:29            1                  N         -73.974754   
2015-06-02 11:19:30            1                  N         -73.978539   
2015-06-02 11:19:31            1                  N         -73.990273   
2015-06-02 11:19:31            1                  N         -73.971527   
2015-06-02 11:19:32            1                  N         -73.982162   
...                          ...                ...                ...   
2015-06-01 00:11:19            1                  N         -73.987930   
2015-06-01 00:11:19            1                  N         -74.001289   
2015-06-01 00:11:19            2                  N         -74.005714   
2015-06-01 00:11:19            1                  N         -73.967545   
2015-06-01 00:11:19            1                  N         -73.942444   

                      dropoff_latitude  payment_type  fare_amount  extra  \
tpep_pickup_datetime                                                       
2015-06-02 11:19:29          40.754093             2         17.0    0.0   
2015-06-02 11:19:30          40.761909             1          6.5    0.0   
2015-06-02 11:19:31          40.745438             1          8.0    0.0   
2015-06-02 11:19:31          40.760330             1         13.5    0.0   
2015-06-02 11:19:32          40.758999             2          9.5    0.0   
...                                ...           ...          ...    ...   
2015-06-01 00:11:19          40.745880             2          7.0    0.5   
2015-06-01 00:11:19          40.736668             1          4.5    0.5   
2015-06-01 00:11:19          40.743641             1         52.0    0.0   
2015-06-01 00:11:19          40.763191             2          5.0    0.5   
2015-06-01 00:11:19          40.845646             2         24.5    0.5   

                      mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                      
2015-06-02 11:19:29       0.5        0.

In [158]:
# regexp also!

df.filter(regex=r'(\d\d):\1:', axis='rows')  

VendorID tpep_dropoff_datetime  passenger_count  \
tpep_pickup_datetime                                                    
2015-06-01 00:00:00          1   2015-06-01 00:06:12                1   
2015-06-01 00:00:02          2   2015-06-01 00:25:48                1   
2015-06-01 00:00:03          2   2015-06-01 00:10:24                1   
2015-06-01 00:00:01          2   2015-06-01 00:24:48                1   
2015-06-01 00:00:02          2   2015-06-01 00:18:57                1   
...                        ...                   ...              ...   
2015-06-04 15:15:57          2   2015-06-04 15:19:55                2   
2015-06-04 15:15:58          1   2015-06-04 15:33:33                1   
2015-06-04 15:15:58          1   2015-06-04 15:42:27                3   
2015-06-04 15:15:59          1   2015-06-04 15:28:57                1   
2015-06-04 15:15:59          2   2015-06-04 15:30:57                1   

                      trip_distance  pickup_longitude  pickup_latitude  \
tpep_pickup_datetime                                                     
2015-06-01 00:00:00            1.00        -73.988739        40.756832   
2015-06-01 00:00:02            5.51        -73.987602        40.753948   
2015-06-01 00:00:03            2.59        -73.971085        40.758762   
2015-06-01 00:00:01            8.15        -74.006844        40.730572   
2015-06-01 00:00:02            9.04        -73.993622        40.746792   
...                             ...               ...              ...   
2015-06-04 15:15:57            0.78        -73.967819        40.792442   
2015-06-04 15:15:58            0.80        -73.978577        40.750118   
2015-06-04 15:15:58            9.10        -73.973434        40.743942   
2015-06-04 15:15:59            1.50        -73.966034        40.794914   
2015-06-04 15:15:59            1.71        -73.989708        40.741772   

                      RateCodeID store_and_fwd_flag  dropoff_longitude  \
tpep_pickup_datetime                                                     
2015-06-01 00:00:00            1                  N         -73.974701   
2015-06-01 00:00:02            1                  N         -73.929169   
2015-06-01 00:00:03            1                  N         -73.950012   
2015-06-01 00:00:01            1                  N         -73.946342   
2015-06-01 00:00:02            1                  N         -73.941841   
...                          ...                ...                ...   
2015-06-04 15:15:57            1                  N         -73.971603   
2015-06-04 15:15:58            1                  N         -73.980331   
2015-06-04 15:15:58            1                  N         -73.861610   
2015-06-04 15:15:59            1                  N         -73.958961   
2015-06-04 15:15:59            1                  N         -74.002327   

                      dropoff_latitude  payment_type  fare_amount  extra  \
tpep_pickup_datetime                                                       
2015-06-01 00:00:00          40.757038             2          6.0    0.5   
2015-06-01 00:00:02          40.770500             2         20.5    0.5   
2015-06-01 00:00:03          40.789864             1         10.5    0.5   
2015-06-01 00:00:01          40.811508             1         26.5    0.5   
2015-06-01 00:00:02          40.843319             1         27.0    0.5   
...                                ...           ...          ...    ...   
2015-06-04 15:15:57          40.797546             2          5.0    0.0   
2015-06-04 15:15:58          40.759350             1         11.0    0.0   
2015-06-04 15:15:58          40.768246             1         28.0    0.0   
2015-06-04 15:15:59          40.780773             1          9.5    0.0   
2015-06-04 15:15:59          40.722965             1         11.0    0.0   

                      mta_tax  tip_amount  tolls_amount  \
tpep_pickup_datetime                                      
2015-06-01 00:00:00       0.5        0.